In [1]:
# Importações necessárias
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *
import pandas as pd
import re


# Função para normalizar os nomes das colunas
def normalizar_coluna(nome):
    # Remove caracteres especiais e troca por underline
    nome = re.sub(r"[^\w]", "_", nome.strip())
    # Substitui múltiplos underlines seguidos por apenas um
    nome = re.sub(r"__+", "_", nome)
    return nome

# Valor padrão para campos monetários vazios
moeda = 0

# Caminho para o arquivo Excel (Google Sheets exportado como .xlsx)
caminho_do_arquivo_excel = "https://docs.google.com/spreadsheets/d/1cucnW4yVosO5n5BFgwXYv6rVy8yj6NTasM83RTCMOug/export?format=xlsx"  # Substitua pelo caminho do seu arquivo Excel

# Lista todas as abas (planilhas) do arquivo
excel_file = pd.ExcelFile(caminho_do_arquivo_excel)

# Obtém os nomes das abas
nomes_abas = excel_file.sheet_names

# Itera por cada aba do Excel
for nome_aba in nomes_abas:

    # Lê os dados da aba correspondente
    df = excel_file.parse(sheet_name=nome_aba)   

     ## TRATAMENTO ABA RECEITA
    if nome_aba == 'Receita':

       # Converte todos os valores para string 
        df_receita = df.astype(str)

        # Normaliza os nomes das colunas
        df_receita.columns = [col.strip().replace("\n", " ")
                                        .replace("(", "")
                                        .replace(")", "")
                                        .replace(" ", "_")
                                        .replace(".", "_")
                                        .replace("-", "_")
                              for col in df_receita.columns]
        # Aplica a função de normalização final
        df_receita.columns = [normalizar_coluna(c) for c in df_receita.columns]

        # Converte para DataFrame Spark
        df_spark = spark.createDataFrame(df_receita)

        # Substitui valores nulos por valor padrão definido (0)
        df_spark = df_spark\
                  .withColumn("Valor_Receita", when(col("Valor_Receita").isNull(), lit(moeda)).otherwise(col("Valor_Receita")))\
                  .withColumn("Valor", when(col("Valor").isNull(), lit(moeda)).otherwise(col("Valor")))
        
        # Define o caminho de salvamento
        path = (  
                f"Files/Abas/"
                f"{nome_aba}"
                f".parquet"
                )

        # Salva como arquivo Parquet
        df_spark.write.mode("overwrite").parquet(path)

  
    
    ## TRATAMENTO ABA DESPESAS    
    if nome_aba == 'Despesas':

        # Lê os dados sem cabeçalho
        df = excel_file.parse(sheet_name=nome_aba, header=None) 
        
        # Converte todos os valores para string
        df_Despesas = df.astype(str)

        # Normaliza os nomes das colunas
        df_Despesas.columns = [str(col).strip().replace("\n", " ")
                                .replace("(", "")
                                .replace(")", "")
                                .replace(" ", "_")
                                .replace(".", "_")
                                .replace("-", "_")
                      for col in df_Despesas.columns]

        # Adiciona índice como coluna para manter ordenação
        df_Despesas = df_Despesas.reset_index(drop=True)  # garante índice sequencial
        df_Despesas['linha_ordem'] = df_Despesas.index  # cria coluna de ordenação

        # Converte para Spark DataFrame
        df_spark = spark.createDataFrame(df_Despesas)

        # Define o caminho de salvamento
        path = (  
                f"Files/Abas/"
                f"{nome_aba}"
                f".parquet"
                )

        # Salva em Parquet 
        df_spark.orderBy("linha_ordem").write.mode("overwrite").parquet(path)
        

    ## TRATAMENTO ABA FINANCEIRO MENSAL 
    if nome_aba == 'Financeiro Mensal':

        # Converte todos os valores para string
        df_Financeiro_Mensal = df.astype(str)

        # Ajuste básico nos nomes das colunas (remove quebras de linha)
        df_Financeiro_Mensal.columns = [str(col).strip().replace("\n", " ") for col in df_Financeiro_Mensal.columns]

        # Cria o DataFrame Spark
        df_spark = spark.createDataFrame(df_Financeiro_Mensal)

        # Define o caminho de salvamento
        path = (  
                f"Files/Abas/"
                f"{nome_aba}"
                f".parquet"
                )

        # Salva em Parquet
        df_spark.write.mode("overwrite").parquet(path)

    
    ## TRATAMENTO ABA FINANCEIRO ANUAL
    if nome_aba == 'Financeiro Anual':

       # Converte todos os valores para string
        df_Financeiro_Anual = df.astype(str)

        # Normaliza os nomes das colunas
        df_Financeiro_Anual.columns = [col.strip().replace("\n", " ")
                                .replace("(", "")
                                .replace(")", "")
                                .replace(" ", "_")
                                .replace(".", "_")
                                .replace("-", "_")
                      for col in df_Financeiro_Anual.columns]

        # Cria o DataFrame Spark
        df_spark = spark.createDataFrame(df_Financeiro_Anual)

        # Define o caminho de salvamento
        path = (  
                f"Files/Abas/"
                f"{nome_aba}"
                f".parquet"
                )

        # Salva em Parquet
        df_spark.write.mode("overwrite").parquet(path)


    ## TRATAMENTO ABA PLR
    if nome_aba == 'PLR':

       # Converte todos os valores para string 
        df_PLR = df.astype(str)

        # Normaliza os nomes das colunas
        df_PLR.columns = [col.strip().replace("\n", " ")
                                .replace("(", "")
                                .replace(")", "")
                                .replace(" ", "_")
                                .replace(".", "_")
                                .replace("-", "_")
                      for col in df_PLR.columns]

        # Cria o DataFrame Spark
        df_spark = spark.createDataFrame(df_PLR)

        # Define o caminho de salvamento
        path = (  
                f"Files/Abas/"
                f"{nome_aba}"
                f".parquet"
                )

        # Salva em Parquet
        df_spark.write.mode("overwrite").parquet(path)
   

StatementMeta(, 3fce9813-ea88-4297-87df-9d1de8066fcd, 3, Finished, Available, Finished)

In [2]:
%%sql
-- Cria ou substitui uma tabela chamada "Despesas" no catálogo do Spark
CREATE OR REPLACE TABLE Despesas
AS
-- Seleciona todos os dados do arquivo Parquet salvo anteriormente na pasta Files/Abas
SELECT *
FROM parquet.`Files/Abas/Despesas.parquet`

StatementMeta(, 3fce9813-ea88-4297-87df-9d1de8066fcd, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
%%sql
-- Cria ou substitui uma tabela chamada "Receita" no catálogo do Spark
CREATE OR REPLACE TABLE Receita
AS
-- Seleciona todos os dados do arquivo Parquet salvo anteriormente na pasta Files/Abas
SELECT *
FROM parquet.`Files/Abas/Receita.parquet`

StatementMeta(, 3fce9813-ea88-4297-87df-9d1de8066fcd, 5, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [4]:
# Lê o arquivo Parquet salvo anteriormente da aba "Financeiro Mensal" e cria um DataFrame Spark
df = spark.read.parquet("Files/Abas/Financeiro Mensal.parquet")

# Cria uma *temporary view* (visão temporária) chamada "Financeiro_Mensal"
# Isso permite que você consulte o DataFrame usando comandos SQL no Spark
df.createOrReplaceTempView("Financeiro_Mensal")

StatementMeta(, 3fce9813-ea88-4297-87df-9d1de8066fcd, 6, Finished, Available, Finished)

In [5]:
# Lê o arquivo Parquet da aba "Financeiro Anual" e cria um DataFrame Spark
df = spark.read.parquet("Files/Abas/Financeiro Anual.parquet")

# Cria ou substitui uma visão temporária chamada "Financeiro_Anual"
# Essa visão permite executar consultas SQL diretamente sobre os dados
df.createOrReplaceTempView("Financeiro_Anual")

StatementMeta(, 3fce9813-ea88-4297-87df-9d1de8066fcd, 7, Finished, Available, Finished)

In [6]:
%%sql
-- Cria ou substitui uma tabela chamada "PLR" no catálogo do Spark
CREATE OR REPLACE TABLE PLR
AS
-- Seleciona todos os dados do arquivo Parquet salvo anteriormente na pasta Files/Abas
SELECT *
FROM parquet.`Files/Abas/PLR.parquet`


StatementMeta(, 3fce9813-ea88-4297-87df-9d1de8066fcd, 8, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [7]:
# Importações necessárias para manipulação com PySpark e datas
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *

# Define um valor padrão para colunas de moeda que estiverem nulas
moeda = 0

# Caminho para o arquivo Excel no OneLake (Microsoft Fabric)
caminho_do_arquivo_excel = "abfss://DataRugas@onelake.dfs.fabric.microsoft.com/Bronze.Lakehouse/Files/Arquivos/Base 2 - Vendas Clientes ConsigCar.xlsx"  # Substitua pelo caminho do seu arquivo Excel

# Carrega o arquivo Excel como um objeto do Pandas
excel_file = pd.ExcelFile(caminho_do_arquivo_excel)

# Extrai os nomes de todas as abas do Excel
nomes_abas = excel_file.sheet_names

# Loop para processar cada aba individualmente
for nome_aba in nomes_abas:
     # Lê os dados da aba atual como um DataFrame Pandas
    df = excel_file.parse(sheet_name=nome_aba)   

    ## TRATAMENTO DA ABA "Vendas"
    if nome_aba == 'Vendas':

       # Converte todos os valores para string (evita erros na hora de criar DataFrame Spark)
        df_Vendas = df.astype(str)

        # Normaliza os nomes das colunas:
        # - Remove quebras de linha, parênteses, espaços e pontuação
        # - Substitui tudo por underscore (_) para padronização
        df_Vendas.columns = [col.strip().replace("\n", " ")
                                .replace("(", "")
                                .replace(")", "")
                                .replace(" ", "_")
                                .replace(".", "_")
                                .replace("-", "_")
                      for col in df_Vendas.columns]

        # Converte o DataFrame Pandas para Spark
        df_spark = spark.createDataFrame(df_Vendas)

        # Define o caminho de destino para salvar em Parquet
        path = (  
                f"Files/Abas/"
                f"{nome_aba}"
                f".parquet"
                )

        # Salva o DataFrame Spark no formato Parquet
        df_spark.write.mode("overwrite").parquet(path)

    ## TRATAMENTO DA ABA "Matriculas_historico"  
    if nome_aba == 'Matriculas_historico':

        # Converte todos os valores para string (evita erros na hora de criar DataFrame Spark)
        df_Matriculas_historico = df.astype(str)

        # Normaliza os nomes das colunas:
        # - Remove quebras de linha, parênteses, espaços e pontuação
        # - Substitui tudo por underscore (_) para padronização
        df_Matriculas_historico.columns = [col.strip().replace("\n", " ")
                                .replace("(", "")
                                .replace(")", "")
                                .replace(" ", "_")
                                .replace(".", "_")
                                .replace("-", "_")
                      for col in df_Matriculas_historico.columns]

        # Converte o DataFrame Pandas para Spark
        df_spark = spark.createDataFrame(df_Matriculas_historico)

        # Define o caminho de destino para salvar em Parquet
        path = (  
                f"Files/Abas/"
                f"{nome_aba}"
                f".parquet"
                )

        # Salva o DataFrame Spark no formato Parquet
        df_spark.write.mode("overwrite").parquet(path)


StatementMeta(, 3fce9813-ea88-4297-87df-9d1de8066fcd, 9, Finished, Available, Finished)

In [8]:
%%sql
CREATE OR REPLACE TABLE Vendas
AS
SELECT *
FROM parquet.`Files/Abas/Vendas.parquet`


StatementMeta(, 3fce9813-ea88-4297-87df-9d1de8066fcd, 10, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [9]:
%%sql
CREATE OR REPLACE TABLE Matriculas_historico
AS
SELECT *
FROM parquet.`Files/Abas/Matriculas_historico.parquet`


StatementMeta(, 3fce9813-ea88-4297-87df-9d1de8066fcd, 11, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [10]:
%%sql
select * from bronze.receita

StatementMeta(, 3fce9813-ea88-4297-87df-9d1de8066fcd, 12, Finished, Available, Finished)

<Spark SQL result set with 314 rows and 8 fields>

In [11]:
%%sql
select * from silver.receita

StatementMeta(, 3fce9813-ea88-4297-87df-9d1de8066fcd, 13, Finished, Available, Finished)

<Spark SQL result set with 314 rows and 8 fields>